# RooTrip 추천 시스템 - V4
### - 사용자가 해시태그별 좋아요를 누른 숫자를 바탕으로 가중

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

df1= pd.read_csv('rootrip-board-sample.csv')
df1['hashtag'] = df1['hashtag'].apply(json.loads)
print(df1['hashtag'])

user_likes = [100, 1200, 200, 0, 0, 1, 0, 0, 2000, 0, 1000, 4, 0, 0]
          # 액티비티,  여행,   바다, 산,  관광, 자연,음식,문화,도시,모험, 휴양, 풍경, 체험, 데이트
tag_mapping = {
    "액티비티": 0,
    "숙박": 1,
    "바다": 2,
    "산": 3,
    "관광": 4,
    "자연": 5,
    "음식": 6,
    "문화": 7,
    "도시": 8,
    "쇼핑": 9,
    "힐링": 10,
    "풍경": 11,
    "체험": 12,
    "카페": 13
}

def calculate_weights(tags, user_likes):
    weights = [user_likes[tag_mapping.get(tag['tag'], 0)] for tag in tags]
    return ' '.join([str(weight) for weight in weights])

# 각 게시물에 대해 가중치를 계산하여 'weights' 열에 저장
df1['weights'] = df1['hashtag'].apply(calculate_weights, user_likes=user_likes)
print(df1)

# CountVectorizer를 사용하여 단어 벡터화
count = CountVectorizer()
count_matrix = count.fit_transform(df1['weights'])

# 코사인 유사도 매트릭스 계산
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 사용자 입력 가중치와 각 행의 가중치 간의 유사도 계산
user_weight_vector = count.transform([' '.join([str(user_likes[i]) for i in range(len(user_likes))])])
print(user_weight_vector)
similarities = cosine_similarity(user_weight_vector, count_matrix)

# 유사도와 게시글 제목을 매핑
similarities_with_titles = list(enumerate(similarities[0]))
similarities_with_titles = sorted(similarities_with_titles, key=lambda x: x[1], reverse=True)

# 가장 유사한 게시글 추출
recommended_posts = []
for i in range(50):  # 상위 3개 게시글 추천
    post_idx = similarities_with_titles[i][0]
    recommended_posts.append((df1['board_id'].iloc[post_idx]))

# 사용자에게 추천 목록을 출력
print("추천 게시글:")
for post_board_id in recommended_posts:
    print(f"(게시글 ID: {post_board_id})")


0          [{'tag': '바다'}, {'tag': '숙박'}, {'tag': '쇼핑'}]
1          [{'tag': '도시'}, {'tag': '숙박'}, {'tag': '힐링'}]
2           [{'tag': '산'}, {'tag': '자연'}, {'tag': '풍경'}]
3          [{'tag': '도시'}, {'tag': '음식'}, {'tag': '숙박'}]
4          [{'tag': '도시'}, {'tag': '숙박'}, {'tag': '문화'}]
                             ...                        
295        [{'tag': '자연'}, {'tag': '쇼핑'}, {'tag': '체험'}]
296    [{'tag': '힐링'}, {'tag': '액티비티'}, {'tag': '자연'}...
297         [{'tag': '자연'}, {'tag': '산'}, {'tag': '음식'}]
298    [{'tag': '음식'}, {'tag': '체험'}, {'tag': '풍경'}, ...
299         [{'tag': '자연'}, {'tag': '쇼핑'}, {'tag': '산'}]
Name: hashtag, Length: 300, dtype: object
      id                                            hashtag  board_id  \
0      1      [{'tag': '바다'}, {'tag': '숙박'}, {'tag': '쇼핑'}]         1   
1      2      [{'tag': '도시'}, {'tag': '숙박'}, {'tag': '힐링'}]         2   
2      3       [{'tag': '산'}, {'tag': '자연'}, {'tag': '풍경'}]         3   
3      4      [{'tag': '도시'}, {'tag': '

In [13]:
import pickle

In [14]:
df1.head(10)

,id,hashtag,board_id,content,weights
0,1,"[{'tag': '바다'}, {'tag': '숙박'}, {'tag': '쇼핑'}]",1,5일간의 해변 휴가를 마치고 돌아왔어요! 친구들과 함께 떠난 이 여행은 정말 잊지 ...,200 1200 0
1,2,"[{'tag': '도시'}, {'tag': '숙박'}, {'tag': '힐링'}]",2,도시 탐험 3일차를 마치고 돌아왔어요! 친구들과 함께 떠난 이 여행은 다양한 경험이...,2000 1200 1000
2,3,"[{'tag': '산'}, {'tag': '자연'}, {'tag': '풍경'}]",3,자유 여행 4일차를 마치고 돌아왔어요! 이번 여행은 미리 정해진 일정 없이 자유롭게...,0 1 4
3,4,"[{'tag': '도시'}, {'tag': '음식'}, {'tag': '숙박'}]",4,도심 속 작은 발견 2일차를 마치고 돌아왔어요! 도심 속에서도 작은 숨은 보석들을 ...,2000 0 1200
4,5,"[{'tag': '도시'}, {'tag': '숙박'}, {'tag': '문화'}]",5,역동적인 도시 라이프 5일간을 즐기고 돌아왔어요! 이번 여행은 도시의 다양한 활동과...,2000 1200 0
5,6,"[{'tag': '관광'}, {'tag': '문화'}, {'tag': '문화'}, ...",6,역사적인 마을 여행 3일 동안을 마치고 돌아왔어요! 이번 여행은 과거의 흔적들을 찾...,0 0 0 0 1200
6,7,"[{'tag': '관광'}, {'tag': '문화'}, {'tag': '자연'}, ...",7,액티브 어드벤처 여행 7일을 마치고 돌아왔어요! 이번 여행은 활기찬 액티비티와 자연...,0 0 1 0 1200
7,8,"[{'tag': '도시'}, {'tag': '카페'}, {'tag': '풍경'}]",8,레트로한 도심 여행 4일간을 마치고 돌아왔어요! 이번 여행은 도심의 레트로한 분위기...,2000 0 4
8,9,"[{'tag': '관광'}, {'tag': '문화'}, {'tag': '자연'}, ...",9,루프탑 풀파티 여행 2일 동안을 즐기고 돌아왔어요! 이번 여행은 도심 속에서의 화려...,0 0 1 0 1200
9,10,"[{'tag': '관광'}, {'tag': '문화'}, {'tag': '자연'}, ...",10,신비로운 섬 여행 3일차를 마치고 돌아왔어요! 이번 여행은 항해로 도달하는 작은 섬...,0 0 1 0 1200


In [16]:
recommend_borad = df1[['hashtag','board_id']].copy()
recommend_borad

,hashtag,board_id
0,"[{'tag': '바다'}, {'tag': '숙박'}, {'tag': '쇼핑'}]",1
1,"[{'tag': '도시'}, {'tag': '숙박'}, {'tag': '힐링'}]",2
2,"[{'tag': '산'}, {'tag': '자연'}, {'tag': '풍경'}]",3
3,"[{'tag': '도시'}, {'tag': '음식'}, {'tag': '숙박'}]",4
4,"[{'tag': '도시'}, {'tag': '숙박'}, {'tag': '문화'}]",5
...,...,...
295,"[{'tag': '자연'}, {'tag': '쇼핑'}, {'tag': '체험'}]",296
296,"[{'tag': '힐링'}, {'tag': '액티비티'}, {'tag': '자연'}...",297
297,"[{'tag': '자연'}, {'tag': '산'}, {'tag': '음식'}]",298
298,"[{'tag': '음식'}, {'tag': '체험'}, {'tag': '풍경'}, ...",299


In [17]:
pickle.dump(recommend_borad, open('board.pickle','wb'))

In [15]:
pickle.dump(cosine_sim, open('cosine_sim.pickle','wb'))